## Задание 

    Написать приложение, которое собирает основные новости с сайта на выбор lenta.ru, news.mail.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:
        название источника;
        наименование новости;
        ссылку на новость;
        дата публикации.
    Сложить собранные данные в БД

Минимум один сайт, максимум - все три


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from lxml import html
import pprint

import pymongo
from pymongo import MongoClient
import datetime

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'}
now = datetime.datetime.now()

In [2]:
# Подключение к локальному серверу MongoDB.
client = MongoClient('127.0.0.1', 27017)
db = client['news']
news = db.news

In [3]:
# lenta

sourse_name = 'https://lenta.ru'
response = requests.get(sourse_name)
dom = html.fromstring(response.text)

item = dom.xpath('//time[@class = "g-time"]/../..')
for el in item:
    url = 'https://lenta.ru' + el.xpath('./a/@href')[0]
    title = el.xpath('./a/text()')[0]
    title = title.replace('\xa0', ' ')
    publication_date = el.xpath('./a/time/@title')[0]
    
    result_lenta_dic = {
        'url': url,
        'title': title, 
        'publication_date': publication_date, 
        'sourse_name': sourse_name
    }
    # добавляем в БД
    news.insert_one(result_lenta_dic)

In [4]:
# yandex

sourse_name = 'https://yandex.ru/news'
rubrics = ['business', 'personal_feed', 'society', 'world', 'sport', 'incident', 'science']
response = requests.get(sourse_name)
dom = html.fromstring(response.text)

for rubric in rubrics:
    item = dom.xpath(f"//a[contains(@href, 'rubric={rubric}') and @class = 'mg-card__link']/ancestor::article")
    for i in item:
        url = i.xpath(".//a[@class = 'mg-card__link']/@href")[0]
        title = i.xpath(".//h2[@class = 'mg-card__title']/text()")[0] 
        title = title.replace('\xa0', ' ')
        # Все новости на странице имеют текущую дату
        publication_date = now.strftime("%d-%m-%Y")
        sourse_name = i.xpath(".//a[@class = 'mg-card__source-link']/text()")[0]
        
        result_yandex_dic = {
            'url': url,
            'title': title, 
            'publication_date': publication_date, 
            'sourse_name': sourse_name
        }
        # добавляем в БД
        news.insert_one(result_yandex_dic)

In [5]:
for x in news.find():
    pprint.pprint(x)

{'_id': ObjectId('6128aba54b5cd53201aeb62b'),
 'publication_date': '27 августа 2021',
 'sourse_name': 'https://lenta.ru',
 'title': 'Врач предрек россиянам пять-шесть новых волн коронавируса',
 'url': 'https://lenta.ru/news/2021/08/27/wavee/'}
{'_id': ObjectId('6128aba54b5cd53201aeb62c'),
 'publication_date': '27 августа 2021',
 'sourse_name': 'https://lenta.ru',
 'title': 'Турция эвакуировала посольство в Кабуле и начала вывод войск из '
          'Афганистана',
 'url': 'https://lenta.ru/news/2021/08/27/turkambevac/'}
{'_id': ObjectId('6128aba54b5cd53201aeb62d'),
 'publication_date': '27 августа 2021',
 'sourse_name': 'https://lenta.ru',
 'title': 'Застреливший женщину в Капитолии полицейский назвал себя '
          '«спасителем жизней»',
 'url': 'https://lenta.ru/news/2021/08/27/savior/'}
{'_id': ObjectId('6128aba54b5cd53201aeb62e'),
 'publication_date': '27 августа 2021',
 'sourse_name': 'https://lenta.ru',
 'title': 'Женщина рассказала о жизни без пупка из-за редкого врожденного '
